<a href="https://colab.research.google.com/github/ankittejyadav/AI/blob/main/Exercises/4-Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using GloVe Embedding

© Data Trainers LLC. GPL v 3.0.

**Author:** Axel Sirota


In this notebook we will leverage Standford's GloVe vectors which is a pretrained embedding on 1.4B Tweets.

Take it easy and pay attention to the main differences with before, and the non-trainable parameters. Finally, check how accurate the results now are.
You can run this lab both locally or in Colab.

- To run in Colab just go to `https://colab.research.google.com`, sign-in and you upload this notebook. Colab has GPU access for free.
- To run locally just run `jupyter notebook` and access the notebook in this lab. You would need to first install the requirements in `requirements.txt`

Follow the instructions. Good luck!


In [1]:
!nvidia-smi

Mon May 13 14:17:51 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
!pip install textblob 'keras-nlp' 'keras-preprocessing' 'gensim==4.2.0' np_utils 'tensorflow-text==2.15'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.3/515.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 61.1 MB/s eta 0:00:00
  Created wheel for np_utils: filename=np_utils-0.6.0-py3-none-any.whl size=56441 sha256=0d7ef7128b317c30967d90f7020c20fa57d2c6061d111a1f9a739120694580b0
  Stored in directory: /root/.cache/pip/wheels/b6/c7/50/2307607f44366dd021209f660045f8d51cb976514d30be7cc7
Successfully built np_utils
  Attempting uninstall: gensim
    Found existing installation: gensim 4.3.2
    Uninstalling gensim-4.3.2:
      Successfully uninstalled gensim-4.3.2


In [4]:
import multiprocessing
import tensorflow as tf
import sys
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda
import np_utils
from tensorflow.keras.utils import to_categorical
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from textblob import TextBlob, Word
from keras_preprocessing.sequence import pad_sequences
from keras.initializers import Constant
import numpy as np
import random
import os
import pandas as pd
import gensim
import warnings
import nltk

TRACE = False
embedding_dim = 100
epochs=2
batch_size = 500
BATCH = True

def set_seeds_and_trace():
  os.environ['PYTHONHASHSEED'] = '0'
  np.random.seed(42)
  tf.random.set_seed(42)
  random.seed(42)
  if TRACE:
    tf.debugging.set_log_device_placement(True)

def set_session_with_gpus_and_cores():
  cores = multiprocessing.cpu_count()
  gpus = len(tf.config.list_physical_devices('GPU'))
  config = tf.compat.v1.ConfigProto( device_count = {'GPU': gpus  , 'CPU': cores} , intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
  sess = tf.compat.v1.Session(config=config)
  tf.compat.v1.keras.backend.set_session(sess)

set_seeds_and_trace()
set_session_with_gpus_and_cores()
warnings.filterwarnings('ignore')
nltk.download('punkt')
textblob_tokenizer = lambda x: TextBlob(x).words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
%%writefile get_data.sh
if [ ! -f yelp.csv ]; then
  wget -O yelp.csv https://www.dropbox.com/s/xds4lua69b7okw8/yelp.csv?dl=0
fi

if [ ! -f glove.6B.100d.txt ]; then
  wget -O glove.6B.100d.txt https://www.dropbox.com/s/dl1vswq2sz5f1ws/glove.6B.100d.txt?dl=0
fi

Writing get_data.sh


In [6]:
!bash get_data.sh

--2024-05-13 14:23:54--  https://www.dropbox.com/s/xds4lua69b7okw8/yelp.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.83.18, 2620:100:601b:18::a27d:812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.83.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/xds4lua69b7okw8/yelp.csv [following]
--2024-05-13 14:23:55--  https://www.dropbox.com/s/raw/xds4lua69b7okw8/yelp.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc83e3c5d71069bd7064c19e4c3a.dl.dropboxusercontent.com/cd/0/inline/CS1ZEyk-p7Hv1Fb6vrYR5w7OcCd7HEJ8bCr6IyTjLLZfhOJHHbdNmPg-ts6vIJYBWLS9TAF6Qm376qIF2EihwmFMY99y0fV8Du-U2kDq0mjND4Lup9irmwHL_Rr9AJ0BxA8wDBXMb4ZayGdUO_4FYBB2/file# [following]
--2024-05-13 14:23:55--  https://uc83e3c5d71069bd7064c19e4c3a.dl.dropboxusercontent.com/cd/0/inline/CS1ZEyk-p7Hv1Fb6vrYR5w7OcCd7HEJ8bCr6IyTjLLZfhOJHHbdNmPg-ts6vIJYBWLS9TAF6Qm376qIF2EihwmFMY99y0fV8Du-U2kDq0mj

In [7]:
path = './yelp.csv'
yelp = pd.read_csv(path)
# Create a new DataFrame that only contains the 5-star and 1-star reviews to have extremes.
yelp_best_worst = yelp[(yelp.stars==5) | (yelp.stars==1)]
X = yelp_best_worst.text
y = yelp_best_worst.stars.map({1:0, 5:1})

In [10]:
corpus = [sentence for sentence in X.values if type(sentence) == str and len(TextBlob(sentence).words) > 3]

In [15]:
path_to_glove_file = "./glove.6B.100d.txt"
with open(path_to_glove_file) as fp:
  lines = fp.readlines()
embeddings_index = {line.split(" ")[0]: np.array(line.split(" ")[1:]).astype(float) for line in lines}
# Construct a function that fills the embedding_index dict for every word in the GloVe file with its coefficients.
# HELP: For that iterate over the Glove file (hint: check that file to view its structure first!), split the word from the numbers, and populate the dictionary with the word and the numbers as a numpy array.
# Hint2: check np.fromstring
# FILL


print("Found %s word vectors." % len(embeddings_index))

Found 400001 word vectors.


In [12]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
tokenized_corpus = tokenizer.texts_to_sequences(corpus)
nb_samples = sum(len(s) for s in corpus)
vocab_size = len(tokenizer.word_index) + 1

In [13]:
print(f'First 5 corpus items are {corpus[:5]}')
print(f'Length of corpus is {len(corpus)}')

First 5 corpus items are ['My wife took me here on my birthday for breakfast and it was excellent.  The weather was perfect which made sitting outside overlooking their grounds an absolute pleasure.  Our waitress was excellent and our food arrived quickly on the semi-busy Saturday morning.  It looked like the place fills up pretty quickly so the earlier you get here the better.\n\nDo yourself a favor and get their Bloody Mary.  It was phenomenal and simply the best I\'ve ever had.  I\'m pretty sure they only use ingredients from their garden and blend them fresh when you order it.  It was amazing.\n\nWhile EVERYTHING on the menu looks excellent, I had the white truffle scrambled eggs vegetable skillet and it was tasty and delicious.  It came with 2 pieces of their griddled bread with was amazing and it absolutely made the meal complete.  It was the best "toast" I\'ve ever had.\n\nAnyway, I can\'t wait to go back!', 'I have no idea why some people give bad reviews about this place. It g

In [16]:
len(embeddings_index['a'])

100

In [17]:
hits = 0
misses = 0

embedding_matrix = np.zeros((vocab_size, embedding_dim))

# Create a loop such that for every word in the vocabulary, if it exists in the Glove embedding, then set for that word (that means that index) the tensor of the Glove Embedding. Otherwise fill with 0
# FILL
for word in tokenizer.word_index:
  if word in embeddings_index:
    hits += 1
    word_index = tokenizer.word_index[word]
    embedding_matrix[word_index] = embeddings_index[word]
  else:
    misses += 1
# In the end, the embedding matrix should have, for every word of our tokenizer that exists in GloVe, the tensor representation of that word

In [18]:
embedding_matrix.shape

(19950, 100)

In [19]:
print(f"Hits: {hits} Misses: {misses}")

Hits: 17190 Misses: 2759


In [20]:
model = Sequential()
embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=4, trainable=False, weights=[embedding_matrix])
model.add(embedding_layer)  # Add the new embedding and set it as non-trainable (although you could fine-tune it if you prefer)
model.add(Lambda(lambda x: tf.reduce_mean(x, axis=1)))  # Add the same Lambda as before to average out the words dimension
model.add(Dense(vocab_size, activation='softmax'))  # Add a Dense layer to classify words

In [21]:
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 100)            1995000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 19950)             2014950   
                                                                 
Total params: 4009950 (15.30 MB)
Trainable params: 2014950 (7.69 MB)
Non-trainable params: 1995000 (7.61 MB)
_________________________________________________________________


Notice the Non-trainable parameters! What we are doing is just training the softmax based on correct embeddings. This is called fine tuning the embedding.


In [25]:
def generate_data(corpus, vocab_size, window_size=2, sentence_batch_size=10,  batch_size=250):
    # np.random.shuffle(np.array(corpus))
    number_of_sentence_batches = (len(corpus) // sentence_batch_size) + 1
    for batch in range(number_of_sentence_batches):
        lower_end = batch*batch_size
        upper_end = (batch+1)*batch_size if batch+1 < number_of_sentence_batches else len(corpus)
        mini_batch_size = upper_end - lower_end
        maxlen = window_size*2
        X = []
        Y = []
        for review_id, words in enumerate(corpus[lower_end:upper_end]):
            L = len(words)
            for index, word in enumerate(words):
                contexts = []
                labels   = []
                s = index - window_size
                e = index + window_size + 1

                contexts.append([words[i] for i in range(s, e) if 0 <= i < L and i != index])
                labels.append(word)

                x = pad_sequences(contexts, maxlen=maxlen)
                y = to_categorical(labels, vocab_size)
                X.append(x)
                Y.append(y)
        X = tf.constant(X)
        Y = tf.constant(Y)
        number_of_batches = len(X) // batch_size
        for real_batch in range(number_of_batches):
          lower_end = real_batch*batch_size
          upper_end = (real_batch+1)*batch_size
          batch_X = tf.squeeze(X[lower_end:upper_end])
          batch_Y = tf.squeeze(Y[lower_end:upper_end])
          yield (batch_X, batch_Y)

In [26]:
# Re implement the method
def fit_model():
    if not BATCH:
        # If we are not batching, Fill how to get X AND Y
        X, Y = None # Fill
        print(f'Size of X is {X.shape} and Y is {Y.shape}')
        model.fit(X, Y, epochs = epochs)
    else:
        # Implement the batching logic to train the model (Hint: use the train_on_batch method of Keras models)
        for epoch in range(epochs):
          batch = 1
          for x, y in generate_data(corpus=tokenized_corpus[:100], vocab_size=vocab_size, batch_size=batch_size):
              print(f'Epoch: {epoch+1}, Batch {batch}')
              history = model.train_on_batch(x, y, return_dict=True)
              batch += 1
              print(f'Loss: {history["loss"]}')

In [27]:
fit_model()

Epoch: 1, Batch 1
Loss: 9.897858619689941
Epoch: 1, Batch 2
Loss: 9.876773834228516
Epoch: 1, Batch 3
Loss: 9.847930908203125
Epoch: 1, Batch 4
Loss: 9.81946086883545
Epoch: 1, Batch 5
Loss: 9.802865982055664
Epoch: 1, Batch 6
Loss: 9.77209186553955
Epoch: 1, Batch 7
Loss: 9.746910095214844
Epoch: 1, Batch 8
Loss: 9.741079330444336
Epoch: 1, Batch 9
Loss: 9.699331283569336
Epoch: 1, Batch 10
Loss: 9.655882835388184
Epoch: 1, Batch 11
Loss: 9.66711139678955
Epoch: 1, Batch 12
Loss: 9.610776901245117
Epoch: 1, Batch 13
Loss: 9.608368873596191
Epoch: 1, Batch 14
Loss: 9.553179740905762
Epoch: 1, Batch 15
Loss: 9.47033977508545
Epoch: 1, Batch 16
Loss: 9.482272148132324
Epoch: 1, Batch 17
Loss: 9.453965187072754
Epoch: 1, Batch 18
Loss: 9.417654037475586
Epoch: 1, Batch 19
Loss: 9.394283294677734
Epoch: 1, Batch 20
Loss: 9.383123397827148
Epoch: 1, Batch 21
Loss: 9.300244331359863
Epoch: 1, Batch 22
Loss: 9.37417221069336
Epoch: 1, Batch 23
Loss: 9.311744689941406
Epoch: 2, Batch 1
Loss: 9

In [28]:
with open('./vectors.txt' ,'w') as f:
    f.write('{} {}\n'.format(vocab_size-1, embedding_dim))
    vectors = model.get_weights()[0]
    for word, i in tokenizer.word_index.items():
        str_vec = ' '.join(map(str, list(vectors[i, :])))
        f.write('{} {}\n'.format(word, str_vec))


In [29]:
w2v = gensim.models.KeyedVectors.load_word2vec_format('./vectors.txt', binary=False)

In [30]:
w2v.most_similar(positive=['pizza'])

[('sandwich', 0.7488481402397156),
 ('sandwiches', 0.7221847772598267),
 ('bread', 0.700667679309845),
 ('pizzas', 0.7004502415657043),
 ('burger', 0.6977595686912537),
 ('snack', 0.6840099692344666),
 ('taco', 0.6788282990455627),
 ('pie', 0.6776915788650513),
 ('chicken', 0.6714670658111572),
 ('burgers', 0.66903156042099)]

In [31]:
w2v.most_similar(positive=['grape'])

[('grapes', 0.7915149927139282),
 ('wine', 0.7132691144943237),
 ('varieties', 0.7115378975868225),
 ('chardonnay', 0.7091464996337891),
 ('wines', 0.7076444029808044),
 ('pinot', 0.7055837512016296),
 ('zinfandel', 0.7001710534095764),
 ('vines', 0.681800127029419),
 ('sauvignon', 0.6581584811210632),
 ('cabernet', 0.6564268469810486)]

Do you notice the difference in the accuracy? For any task first search if there are any pretrained models to use!